In [59]:
1+1

2

In [60]:
pip install streamlit python-dotenv langchain-openai faiss-cpu pymupdf langchain-community numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [61]:
from dotenv import load_dotenv

load_dotenv()

True

In [62]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", streaming=True)

In [63]:
import fitz  # pymupdf
import os

def extract_text_with_fitz(pdf_path):
    """PyMuPDF로 텍스트 추출 - 한글 처리 개선"""
    try:
        doc = fitz.open(pdf_path)
        print(f"PDF 파일: {pdf_path}")
        print(f"총 페이지 수: {len(doc)}")
        
        full_text = ""
        
        for page_num in range(len(doc)):
            print(f"페이지 {page_num + 1} 처리 중...")
            page = doc[page_num]
            
            # 텍스트 추출
            text = page.get_text()
            
            if text and text.strip():
                # 한글 처리 및 텍스트 정리
                text = text.replace('\x00', '')  # null 문자 제거
                text = text.replace('\ufeff', '')  # BOM 제거
                text = text.replace('\r\n', '\n')  # 줄바꿈 정리
                text = text.replace('\r', '\n')
                
                print(f"  페이지 {page_num + 1}: {len(text)} 글자 추출")
                print(f"  첫 50글자: {text[:50]}")
                
                full_text += text + "\n\n"  # 페이지 구분
            else:
                print(f"  페이지 {page_num + 1}: 텍스트 없음")
        
        doc.close()
        print(f"\n총 추출된 텍스트: {len(full_text)} 글자")
        return full_text
        
    except Exception as e:
        print(f"PyMuPDF 오류: {e}")
        return ""

# 텍스트 추출
pdf_file = "2024-PR-15.pdf"
full_text = extract_text_with_fitz(pdf_file)

if full_text:
    print("✓ 텍스트 추출 성공!")
    print(f"전체 텍스트 첫 200글자:")
    print(full_text[:300])
else:
    print("✗ 텍스트 추출 실패")

PDF 파일: 2024-PR-15.pdf
총 페이지 수: 127
페이지 1 처리 중...
  페이지 1: 61 글자 추출
  첫 50글자: 약자와 동행하는 서울의 교통
이신해   김승준   한영준   양재환 
윤서연   연준형  
페이지 2 처리 중...
  페이지 2: 16 글자 추출
  첫 50글자: 약자와 동행하는 서울의 교통

페이지 3 처리 중...
  페이지 3: 257 글자 추출
  첫 50글자: 이 보고서의 내용은 연구진의 견해로서 
서울특별시의 정책과는 다를 수도 있습니다.
연구책임
페이지 4 처리 중...
  페이지 4: 1010 글자 추출
  첫 50글자: i
약
자
와
 동
행
하
는
 서
울
의
 교
통
서울시, 다양한 교통약자 불편 개선 위
페이지 5 처리 중...
  페이지 5: 979 글자 추출
  첫 50글자: ii
요
약
서울 장시간 통근자 지원 시급…혼잡 완화·서비스 수준 보장 방안 필요
교통 체
페이지 6 처리 중...
  페이지 6: 321 글자 추출
  첫 50글자: iii
약
자
와
 동
행
하
는
 서
울
의
 교
통
목차
01 연구개요
2
1_연구배경
페이지 7 처리 중...
  페이지 7: 649 글자 추출
  첫 50글자: iv
목
차
표목차
[표 2-1] 교통부문에서의 약자 정의와 권리, 책무 
14
[표 3-
페이지 8 처리 중...
  페이지 8: 752 글자 추출
  첫 50글자: v
약
자
와
 동
행
하
는
 서
울
의
 교
통
그림목차
[그림 1-1] 약자의 범위 
페이지 9 처리 중...
  페이지 9: 1125 글자 추출
  첫 50글자: vi
목
차
[그림 3-15] 고령자가 걸어서 동네 외출 시 가장 불편한 점
38
[그림 
페이지 10 처리 중...
  페이지 10: 820 글자 추출
  첫 50글자: vii
약
자
와
 동
행
하
는
 서
울
의
 교
통
[그림 4-24] 현재 거주지 선택
페이지 11 처리 중...
  페이지 11: 32 글자 추출
  첫 50글자: 01
연구개요
1_연구

In [64]:
def chunk_text(text, chunk_size=2000, overlap=100):
    """텍스트를 청크로 나누는 함수"""
    if not text.strip():
        print("빈 텍스트입니다.")
        return []
    
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end].strip()
        
        if chunk:  # 빈 청크가 아닌 경우만 추가
            chunks.append(chunk)
        
        start = end - overlap
    
    return chunks

# 텍스트 청킹
if full_text:
    chunks = chunk_text(full_text)
    
    print(f"\n=== 청킹 결과 ===")
    print(f"총 청크 개수: {len(chunks)}")
    
    if chunks:
        chunk_lengths = [len(chunk) for chunk in chunks]
        print(f"청크 길이 - 최소: {min(chunk_lengths)}, 최대: {max(chunk_lengths)}, 평균: {sum(chunk_lengths)/len(chunk_lengths):.1f}")
        
        print(f"\n첫 번째 청크 미리보기:")
        print(chunks[0][:200] + "..." if len(chunks[0]) > 200 else chunks[0])
        
        print(f"\n마지막 청크 미리보기:")
        print(chunks[-1][:200] + "..." if len(chunks[-1]) > 200 else chunks[-1])


=== 청킹 결과 ===
총 청크 개수: 49
청크 길이 - 최소: 129, 최대: 2000, 평균: 1961.4

첫 번째 청크 미리보기:
약자와 동행하는 서울의 교통
이신해   김승준   한영준   양재환 
윤서연   연준형   김영범   김지한


약자와 동행하는 서울의 교통


이 보고서의 내용은 연구진의 견해로서 
서울특별시의 정책과는 다를 수도 있습니다.
연구책임
이신해 서울연구원 스마트교통연구실 선임연구위원
연구진
김승준 서울연구원 스마트교통연구실 선임연구위원
한영준 서울연구원 스마...

마지막 청크 미리보기:
-15
발행인  오균
발행일  2025년 5월 30일
발행처  서울연구원
ISBN  979-11-5700-908-4 95350  비매품
06756 서울특별시 서초구 남부순환로 340길 57
이 출판물의 판권은 서울연구원에 속합니다.


In [65]:
import faiss
import numpy as np
from langchain_openai import OpenAIEmbeddings
import pickle
import os
from datetime import datetime

# OpenAI API 키 설정 (환경변수에서 읽어오거나 직접 설정)
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY  # 필요한 경우

def save_vectorstore_openai(index, chunks, metadatas, pdf_filename, save_name="pdf_openai_vectors"):
    """벡터 스토어 저장"""
    try:
        # FAISS 인덱스 저장
        faiss.write_index(index, f'{save_name}_vectors.index')
        
        # 청크와 메타데이터 저장
        with open(f'{save_name}_data.pkl', 'wb') as f:
            pickle.dump({
                'chunks': chunks,
                'metadatas': metadatas,
                'pdf_filename': pdf_filename,
                'chunk_count': len(chunks),
                'embedding_model': 'text-embedding-3-large',
                'embedding_type': 'openai'
            }, f)
        
        return True
        
    except Exception as e:
        print(f"저장 오류: {e}")
        return False

def create_faiss_vectorstore_openai(chunks, pdf_filename, save_name="pdf_openai_vectors"):
    """청크들을 FAISS 벡터 스토어로 변환하고 자동 저장 (OpenAI 임베딩 사용)"""
    if not chunks:
        print("청크가 없습니다.")
        return None, None, None, None
    
    print("OpenAI 임베딩 모델 로드 중...")
    embedding = OpenAIEmbeddings(model='text-embedding-3-large')
    
    print(f"임베딩 생성 중... ({len(chunks)}개 청크)")
    print("OpenAI API 호출 중이므로 시간이 걸릴 수 있습니다...")
    
    try:
        # 청크들을 임베딩으로 변환
        embeddings = embedding.embed_documents(chunks)
        embeddings = np.array(embeddings).astype('float32')
        
        print(f"임베딩 완료! 차원: {embeddings.shape[1]}, 개수: {embeddings.shape[0]}")
        
        # FAISS 인덱스 생성
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)  # L2 거리 기반 인덱스
        index.add(embeddings)
        
        # 메타데이터 생성
        metadatas = []
        for i, chunk in enumerate(chunks):
            metadatas.append({
                'chunk_id': i,
                'source': pdf_filename,
                'chunk_size': len(chunk),
                'preview': chunk[:100] + "..." if len(chunk) > 100 else chunk
            })
        
        print(f"FAISS 벡터 스토어 생성 완료! {index.ntotal}개 벡터 저장됨")
        
        # 자동 저장
        print("벡터 스토어를 자동 저장 중...")
        if save_vectorstore_openai(index, chunks, metadatas, pdf_filename, save_name):
            print("✓ 자동 저장 완료!")
        else:
            print("⚠️  자동 저장 실패")
        
        return index, embedding, chunks, metadatas
        
    except Exception as e:
        print(f"임베딩 생성 오류: {e}")
        print("OpenAI API 키가 설정되어 있는지 확인해주세요.")
        return None, None, None, None

def load_or_create_vectorstore(chunks, pdf_filename, save_name="pdf_openai_vectors"):
    """
    벡터 스토어가 있으면 로드하고, 없으면 새로 생성하는 함수
    """
    index_file = f'{save_name}_vectors.index'
    data_file = f'{save_name}_data.pkl'
    
    # 저장된 파일들이 모두 존재하는지 확인
    if os.path.exists(index_file) and os.path.exists(data_file):
        print("기존 벡터 스토어를 찾았습니다. 로드 중...")
        
        try:
            # 저장된 데이터 로드
            index = faiss.read_index(index_file)
            
            with open(data_file, 'rb') as f:
                data = pickle.load(f)
            
            # 저장된 PDF 파일명과 현재 파일명 비교
            if data['pdf_filename'] == pdf_filename:
                print("✓ 동일한 PDF 파일의 벡터 스토어 로드 성공!")
                
                # OpenAI 임베딩 모델 로드
                embedding_model = OpenAIEmbeddings(model='text-embedding-3-large')
                
                print(f"- 벡터 개수: {index.ntotal}")
                print(f"- 청크 개수: {data['chunk_count']}")
                print(f"- 원본 파일: {data['pdf_filename']}")
                print(f"- 임베딩 모델: {data.get('embedding_model', '정보 없음')}")
                
                return index, embedding_model, data['chunks'], data['metadatas']
            else:
                print(f"⚠️  다른 PDF 파일의 벡터 스토어입니다.")
                print(f"   저장된 파일: {data['pdf_filename']}")
                print(f"   현재 파일: {pdf_filename}")
                print("   새로운 벡터 스토어를 생성합니다...")
                
        except Exception as e:
            print(f"⚠️  벡터 스토어 로드 중 오류 발생: {e}")
            print("   새로운 벡터 스토어를 생성합니다...")
    
    else:
        print("저장된 벡터 스토어를 찾을 수 없습니다. 새로 생성합니다...")
    
    # 새로운 벡터 스토어 생성
    print("\n=== 새로운 벡터 스토어 생성 ===")
    return create_faiss_vectorstore_openai(chunks, pdf_filename, save_name)

# FAISS 벡터 스토어 자동 로드/생성
if 'chunks' in locals() and chunks:
    print("=== 벡터 스토어 처리 ===")
    index, embedding_model, chunks, metadatas = load_or_create_vectorstore(
        chunks, 
        pdf_file, 
        save_name="my_pdf_vectors"  # 원하는 저장 이름으로 변경 가능
    )
    
    if index is not None:
        print("✓ 벡터 스토어 준비 완료!")
        print("이제 검색을 할 수 있습니다!")
    else:
        print("✗ 벡터 스토어 준비 실패")
else:
    print("청크가 없습니다. 먼저 텍스트 추출과 청킹을 실행해주세요.")

=== 벡터 스토어 처리 ===
기존 벡터 스토어를 찾았습니다. 로드 중...
✓ 동일한 PDF 파일의 벡터 스토어 로드 성공!
- 벡터 개수: 49
- 청크 개수: 49
- 원본 파일: 2024-PR-15.pdf
- 임베딩 모델: text-embedding-3-large
✓ 벡터 스토어 준비 완료!
이제 검색을 할 수 있습니다!


In [66]:
# %pip install --upgrade langgraph>=0.3.1

In [67]:
# # 필요한 라이브러리들
# from langgraph.graph import StateGraph, END
# from langgraph.prebuilt import Toolnode
# from langchain_core.tools import tool
# from langchain_openai import ChatOpenAI
# from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
# from typing import TypedDict, List, Dict, Any
# import json

from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode  # ✅ ToolExecutor 대신 ToolNode 사용
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from typing import TypedDict, List, Dict, Any
import numpy as np

In [68]:
# pip install langgraph langchain-core langchain-community

In [69]:
class AgentState(TypedDict):
    messages: List[Any]
    user_query: str
    search_results: List[Dict]
    summary: str
    tool_calls: List[Dict]

In [70]:
def initialize_session_state():
    if 'messages' not in st.session_state:
        st.session_state.messages = []
    if 'file_processed' not in st.session_state:
        st.session_state.file_processed = False
    if 'chunks' not in st.session_state:
        st.session_state.chunks = []
    if 'index' not in st.session_state:
        st.session_state.index = None
    if 'embedding_model' not in st.session_state:
        st.session_state.embedding_model = None
    if 'metadatas' not in st.session_state:
        st.session_state.metadatas = []
    if 'agent_graph' not in st.session_state:
        st.session_state.agent_graph = None

In [71]:
def extract_text_with_fitz(pdf_path):
    """PyMuPDF로 텍스트 추출"""
    try:
        doc = fitz.open(pdf_path)
        full_text = ""
        
        for page_num in range(len(doc)):
            page = doc[page_num]
            text = page.get_text()
            
            if text and text.strip():
                text = text.replace('\x00', '')
                text = text.replace('\ufeff', '')
                text = text.replace('\r\n', '\n')
                text = text.replace('\r', '\n')
                full_text += text + "\n\n"
        
        doc.close()
        return full_text
        
    except Exception as e:
        st.error(f"PDF 텍스트 추출 오류: {e}")
        return ""

def chunk_text(text, chunk_size=2000, overlap=100):
    """텍스트를 청크로 나누는 함수"""
    if not text.strip():
        return []
    
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end].strip()
        
        if chunk:
            chunks.append(chunk)
        
        start = end - overlap
    
    return chunks



In [72]:
# 새 셀 추가 (cell [28])
import numpy as np

def search_documents_openai(query, index, embedding_model, chunks, metadatas, k=3):
    """
    OpenAI 임베딩을 사용한 문서 검색
    """
    try:
        # 쿼리를 임베딩으로 변환
        query_embedding = embedding_model.embed_query(query)
        query_embedding = np.array([query_embedding]).astype('float32')
        
        # FAISS로 유사한 문서 검색
        distances, indices = index.search(query_embedding, k)
        
        # 결과 포맷팅
        results = []
        for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
            if idx < len(chunks):  # 유효한 인덱스인지 확인
                results.append({
                    'chunk_id': idx,
                    'content': chunks[idx],
                    'score': 1.0 - (distance / 2.0),  # 거리를 유사도로 변환
                    'metadata': metadatas[idx] if idx < len(metadatas) else {}
                })
        
        return results
        
    except Exception as e:
        print(f"검색 오류: {e}")
        return []

In [73]:
import faiss
import numpy as np
from langchain_openai import OpenAIEmbeddings
import pickle
import os
from datetime import datetime

# OpenAI API 키 설정 (환경변수에서 읽어오거나 직접 설정)
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY  # 필요한 경우

def save_vectorstore_openai(index, chunks, metadatas, pdf_filename, save_name="pdf_openai_vectors"):
    """벡터 스토어 저장"""
    try:
        # FAISS 인덱스 저장
        faiss.write_index(index, f'{save_name}_vectors.index')
        
        # 청크와 메타데이터 저장
        with open(f'{save_name}_data.pkl', 'wb') as f:
            pickle.dump({
                'chunks': chunks,
                'metadatas': metadatas,
                'pdf_filename': pdf_filename,
                'chunk_count': len(chunks),
                'embedding_model': 'text-embedding-3-large',
                'embedding_type': 'openai'
            }, f)
        
        return True
        
    except Exception as e:
        print(f"저장 오류: {e}")
        return False

def create_faiss_vectorstore_openai(chunks, pdf_filename, save_name="pdf_openai_vectors"):
    """청크들을 FAISS 벡터 스토어로 변환하고 자동 저장 (OpenAI 임베딩 사용)"""
    if not chunks:
        print("청크가 없습니다.")
        return None, None, None, None
    
    print("OpenAI 임베딩 모델 로드 중...")
    embedding = OpenAIEmbeddings(model='text-embedding-3-large')
    
    print(f"임베딩 생성 중... ({len(chunks)}개 청크)")
    print("OpenAI API 호출 중이므로 시간이 걸릴 수 있습니다...")
    
    try:
        # 청크들을 임베딩으로 변환
        embeddings = embedding.embed_documents(chunks)
        embeddings = np.array(embeddings).astype('float32')
        
        print(f"임베딩 완료! 차원: {embeddings.shape[1]}, 개수: {embeddings.shape[0]}")
        
        # FAISS 인덱스 생성
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)  # L2 거리 기반 인덱스
        index.add(embeddings)
        
        # 메타데이터 생성
        metadatas = []
        for i, chunk in enumerate(chunks):
            metadatas.append({
                'chunk_id': i,
                'source': pdf_filename,
                'chunk_size': len(chunk),
                'preview': chunk[:100] + "..." if len(chunk) > 100 else chunk
            })
        
        print(f"FAISS 벡터 스토어 생성 완료! {index.ntotal}개 벡터 저장됨")
        
        # 자동 저장
        print("벡터 스토어를 자동 저장 중...")
        if save_vectorstore_openai(index, chunks, metadatas, pdf_filename, save_name):
            print("✓ 자동 저장 완료!")
        else:
            print("⚠️  자동 저장 실패")
        
        return index, embedding, chunks, metadatas
        
    except Exception as e:
        print(f"임베딩 생성 오류: {e}")
        print("OpenAI API 키가 설정되어 있는지 확인해주세요.")
        return None, None, None, None

def load_or_create_vectorstore(chunks, pdf_filename, save_name="pdf_openai_vectors"):
    """
    벡터 스토어가 있으면 로드하고, 없으면 새로 생성하는 함수
    """
    index_file = f'{save_name}_vectors.index'
    data_file = f'{save_name}_data.pkl'
    
    # 저장된 파일들이 모두 존재하는지 확인
    if os.path.exists(index_file) and os.path.exists(data_file):
        print("기존 벡터 스토어를 찾았습니다. 로드 중...")
        
        try:
            # 저장된 데이터 로드
            index = faiss.read_index(index_file)
            
            with open(data_file, 'rb') as f:
                data = pickle.load(f)
            
            # 저장된 PDF 파일명과 현재 파일명 비교
            if data['pdf_filename'] == pdf_filename:
                print("✓ 동일한 PDF 파일의 벡터 스토어 로드 성공!")
                
                # OpenAI 임베딩 모델 로드
                embedding_model = OpenAIEmbeddings(model='text-embedding-3-large')
                
                print(f"- 벡터 개수: {index.ntotal}")
                print(f"- 청크 개수: {data['chunk_count']}")
                print(f"- 원본 파일: {data['pdf_filename']}")
                print(f"- 임베딩 모델: {data.get('embedding_model', '정보 없음')}")
                
                return index, embedding_model, data['chunks'], data['metadatas']
            else:
                print(f"⚠️  다른 PDF 파일의 벡터 스토어입니다.")
                print(f"   저장된 파일: {data['pdf_filename']}")
                print(f"   현재 파일: {pdf_filename}")
                print("   새로운 벡터 스토어를 생성합니다...")
                
        except Exception as e:
            print(f"⚠️  벡터 스토어 로드 중 오류 발생: {e}")
            print("   새로운 벡터 스토어를 생성합니다...")
    
    else:
        print("저장된 벡터 스토어를 찾을 수 없습니다. 새로 생성합니다...")
    
    # 새로운 벡터 스토어 생성
    print("\n=== 새로운 벡터 스토어 생성 ===")
    return create_faiss_vectorstore_openai(chunks, pdf_filename, save_name)

# FAISS 벡터 스토어 자동 로드/생성
if 'chunks' in locals() and chunks:
    print("=== 벡터 스토어 처리 ===")
    index, embedding_model, chunks, metadatas = load_or_create_vectorstore(
        chunks, 
        pdf_file, 
        save_name="my_pdf_vectors"  # 원하는 저장 이름으로 변경 가능
    )
    
    if index is not None:
        print("✓ 벡터 스토어 준비 완료!")
        print("이제 검색을 할 수 있습니다!")
    else:
        print("✗ 벡터 스토어 준비 실패")
else:
    print("청크가 없습니다. 먼저 텍스트 추출과 청킹을 실행해주세요.")

=== 벡터 스토어 처리 ===
기존 벡터 스토어를 찾았습니다. 로드 중...
✓ 동일한 PDF 파일의 벡터 스토어 로드 성공!
- 벡터 개수: 49
- 청크 개수: 49
- 원본 파일: 2024-PR-15.pdf
- 임베딩 모델: text-embedding-3-large
✓ 벡터 스토어 준비 완료!
이제 검색을 할 수 있습니다!


In [74]:
# cell [30] 교체
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import tempfile
import os

# 웹 검색 도구 초기화
try:
    search_tool = DuckDuckGoSearchRun()
    print("✅ 웹 검색 도구 준비 완료")
except Exception as e:
    print(f"⚠️ 웹 검색 도구 초기화 실패: {e}")
    search_tool = None

@tool
def search_uploaded_documents(query: str, k: int = 3) -> str:
    """
    업로드된 문서에서 내용을 검색합니다.
    """
    global index, embedding_model, chunks, metadatas
    
    if 'index' not in globals() or index is None:
        return "업로드된 문서가 없습니다. 먼저 파일을 업로드해주세요."
    
    try:
        results = search_documents_openai(query, index, embedding_model, chunks, metadatas, k)
        
        if not results:
            return f"'{query}'에 대한 검색 결과가 없습니다."
        
        formatted_results = []
        for i, result in enumerate(results):
            formatted_results.append(
                f"[문서 검색결과 {i+1}]\n"
                f"유사도: {result['score']:.3f}\n"
                f"내용: {result['content'][:400]}{'...' if len(result['content']) > 400 else ''}\n"
            )
        
        return "\n\n".join(formatted_results)
        
    except Exception as e:
        return f"문서 검색 중 오류: {str(e)}"

@tool
def web_search(query: str) -> str:
    """
    웹에서 최신 정보를 검색합니다.
    """
    if search_tool is None:
        return "웹 검색 기능을 사용할 수 없습니다."
    
    try:
        results = search_tool.run(query)
        return f"🌐 웹 검색 결과:\n{results}"
    except Exception as e:
        return f"웹 검색 중 오류 발생: {str(e)}"

@tool
def summarize_content(content: str, style: str = "general") -> str:
    """
    주어진 내용을 요약합니다.
    """
    if not content.strip():
        return "요약할 내용이 없습니다."
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
    style_prompts = {
        "general": "다음 내용을 자연스럽게 요약해주세요:",
        "bullet_points": "다음 내용을 주요 포인트별로 불릿 포인트로 요약해주세요:",
        "executive": "다음 내용을 경영진을 위한 간단한 요약으로 작성해주세요:",
        "detailed": "다음 내용을 상세하게 요약하되 중요한 내용은 빠뜨리지 마세요:"
    }
    
    prompt = style_prompts.get(style, style_prompts["general"])
    
    try:
        response = llm.invoke([
            HumanMessage(content=f"{prompt}\n\n{content}")
        ])
        return response.content
    except Exception as e:
        return f"요약 중 오류 발생: {str(e)}"

@tool
def get_document_info() -> str:
    """
    현재 로드된 PDF 문서의 정보를 반환합니다.
    """
    global chunks, metadatas, pdf_file
    
    if 'chunks' not in globals() or not chunks:
        return "로드된 문서가 없습니다."
    
    total_chars = sum(len(chunk) for chunk in chunks)
    
    return f"""
📄 현재 로드된 문서 정보:
- 파일명: {pdf_file if 'pdf_file' in globals() else '알 수 없음'}
- 총 청크 수: {len(chunks)}
- 총 글자 수: {total_chars:,}
- 평균 청크 크기: {total_chars // len(chunks):,} 글자

🛠️ 사용 가능한 기능:
- 문서 검색: "교통약자에 대해 찾아줘"
- 웹 검색: "최신 AI 뉴스 검색해줘"  
- 내용 요약: "이 문서를 요약해줘"
"""

✅ 웹 검색 도구 준비 완료


In [75]:
# 새 셀 추가
from langgraph.prebuilt import ToolNode

def enhanced_call_model(state: AgentState) -> AgentState:
    """향상된 LLM 모델 호출"""
    messages = state["messages"]
    
    system_prompt = """
당신은 다기능 AI 어시스턴트입니다. 다음 기능들을 제공할 수 있습니다:

🔍 **검색 기능**:
- search_uploaded_documents: 업로드된 PDF 문서 검색
- web_search: 최신 웹 정보 검색

📝 **분석 기능**:
- summarize_content: 다양한 스타일로 내용 요약
- get_document_info: 현재 문서 상태 확인

사용자 요청에 따라 적절한 도구를 선택하세요:

1. 업로드된 문서에 대한 질문 → search_uploaded_documents 사용
2. 최신 정보나 일반 질문 → web_search 사용  
3. 요약 요청 → summarize_content 사용
4. 문서 정보 요청 → get_document_info 사용

항상 한국어로 친절하고 정확하게 답변해주세요.
"""
    
    if not messages or "system" not in str(messages[0]):
        messages = [HumanMessage(content=system_prompt)] + messages
    
    # 모든 도구 바인딩
    llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
    tools = [search_uploaded_documents, web_search, summarize_content, get_document_info]
    llm_with_tools = llm.bind_tools(tools)
    
    response = llm_with_tools.invoke(messages)
    
    return {"messages": messages + [response]}

# 새로운 도구 노드
tools = [search_uploaded_documents, web_search, summarize_content, get_document_info]
enhanced_tool_node = ToolNode(tools)

In [76]:
@tool
def get_document_info() -> str:
    """
    현재 로드된 PDF 문서의 정보를 반환합니다.
    
    Returns:
        문서 정보
    """
    global chunks, metadatas, pdf_file
    
    if 'chunks' not in globals() or not chunks:
        return "로드된 문서가 없습니다."
    
    total_chars = sum(len(chunk) for chunk in chunks)
    
    return f"""
현재 로드된 문서 정보:
- 파일명: {pdf_file if 'pdf_file' in globals() else '알 수 없음'}
- 총 청크 수: {len(chunks)}
- 총 글자 수: {total_chars:,}
- 평균 청크 크기: {total_chars // len(chunks):,} 글자
"""

In [77]:
def should_continue(state: AgentState) -> str:
    """다음 단계를 결정하는 함수"""
    messages = state["messages"]
    last_message = messages[-1]
    
    # 도구 호출이 있으면 도구 실행
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    
    # 그렇지 않으면 종료
    return END

def call_model(state: AgentState) -> AgentState:
    """LLM 모델 호출"""
    messages = state["messages"]
    
    # 시스템 프롬프트
    system_prompt = """
당신은 PDF 문서 검색 및 요약 전문 에이전트입니다.

사용 가능한 도구들:
1. search_pdf_documents: PDF에서 관련 내용 검색
2. summarize_content: 내용 요약
3. get_document_info: 문서 정보 조회

사용자의 질문에 따라 적절한 도구를 사용하여 답변하세요:
- 특정 내용을 찾고 싶다면 search_pdf_documents를 사용
- 검색된 내용을 요약하고 싶다면 summarize_content를 사용
- 문서 정보가 궁금하다면 get_document_info를 사용

항상 한국어로 친절하고 정확하게 답변해주세요.
"""
    
    # 시스템 메시지가 없다면 추가
    if not messages or not any(isinstance(msg, HumanMessage) for msg in messages[:1]):
        messages = [HumanMessage(content=system_prompt)] + messages
    
    # LLM 호출
    llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
    llm_with_tools = llm.bind_tools([search_pdf_documents, summarize_content, get_document_info])
    
    response = llm_with_tools.invoke(messages)
    
    return {"messages": messages + [response]}

def call_tools(state: AgentState) -> AgentState:
    """도구 실행"""
    messages = state["messages"]
    last_message = messages[-1]
    
    # 도구 실행기 생성
    tools = [search_pdf_documents, summarize_content, get_document_info]
    tool_executor = ToolExecutor(tools)
    
    tool_messages = []
    
    # 각 도구 호출 실행
    for tool_call in last_message.tool_calls:
        try:
            result = tool_executor.invoke(tool_call)
            tool_messages.append(
                ToolMessage(
                    content=str(result),
                    tool_call_id=tool_call["id"]
                )
            )
        except Exception as e:
            tool_messages.append(
                ToolMessage(
                    content=f"도구 실행 오류: {str(e)}",
                    tool_call_id=tool_call["id"]
                )
            )
    
    return {"messages": messages + tool_messages}

In [78]:
def create_agent_graph():
    """LangGraph 에이전트 그래프 생성"""
    try:
        # 도구들
        tools = [search_uploaded_documents, web_search_tool, summarize_content_tool, get_document_info_tool]
        
        # ToolNode 사용 (최신 LangGraph)
        tool_node = ToolNode(tools)
        
        # 그래프 빌더
        graph_builder = StateGraph(AgentState)
        
        # 노드 추가
        graph_builder.add_node("agent", call_model)
        graph_builder.add_node("tools", tool_node)
        
        # 시작점 설정
        graph_builder.set_entry_point("agent")
        
        # 조건부 엣지 추가
        graph_builder.add_conditional_edges(
            "agent",
            should_continue,
            {
                "tools": "tools",
                END: END
            }
        )
        
        # 도구에서 다시 에이전트로
        graph_builder.add_edge("tools", "agent")
        
        # 그래프 컴파일
        return graph_builder.compile()
        
    except Exception as e:
        st.error(f"에이전트 그래프 생성 오류: {e}")
        return None

def run_agent_with_graph(user_query: str) -> str:
    """LangGraph 에이전트 실행"""
    try:
        # 그래프가 없으면 생성
        if st.session_state.agent_graph is None:
            st.session_state.agent_graph = create_agent_graph()
        
        if st.session_state.agent_graph is None:
            return "에이전트 그래프 생성에 실패했습니다."
        
        # 초기 상태 설정
        initial_state = {
            "messages": [HumanMessage(content=user_query)]
        }
        
        # 에이전트 실행
        result = st.session_state.agent_graph.invoke(initial_state)
        final_message = result["messages"][-1]
        
        if hasattr(final_message, 'content'):
            return final_message.content
        else:
            return str(final_message)
            
    except Exception as e:
        return f"에이전트 실행 중 오류: {str(e)}"


In [79]:
# 파일 처리 함수
def process_file(uploaded_file):
    """업로드된 파일 처리"""
    if uploaded_file is None:
        return False, "파일을 업로드해주세요."
    
    if not uploaded_file.name.endswith('.pdf'):
        return False, "PDF 파일만 지원됩니다."
    
    try:
        # 임시 파일에 저장
        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
            tmp_file.write(uploaded_file.getvalue())
            tmp_path = tmp_file.name
        
        # 텍스트 추출
        with st.spinner("PDF 텍스트 추출 중..."):
            full_text = extract_text_with_fitz(tmp_path)
        
        # 임시 파일 삭제
        os.unlink(tmp_path)
        
        if not full_text or len(full_text.strip()) < 10:
            return False, "파일에서 텍스트를 추출할 수 없었습니다."
        
        # 텍스트 청킹
        with st.spinner("텍스트 청킹 중..."):
            chunks = chunk_text(full_text)
        
        if not chunks:
            return False, "텍스트 청킹에 실패했습니다."
        
        # 벡터 스토어 생성
        index, embedding_model, metadatas = create_vectorstore(chunks)
        
        if index is None:
            return False, "벡터 스토어 생성에 실패했습니다."
        
        # 세션 상태 업데이트
        st.session_state.chunks = chunks
        st.session_state.index = index
        st.session_state.embedding_model = embedding_model
        st.session_state.metadatas = metadatas
        st.session_state.file_processed = True
        
        # 기존 그래프 초기화 (새 문서용)
        st.session_state.agent_graph = None
        
        return True, f"파일 처리 완료! 총 {len(chunks)}개 청크로 분할되었습니다."
        
    except Exception as e:
        return False, f"파일 처리 중 오류 발생: {str(e)}"

In [80]:
# 향상된 그래프 생성
enhanced_graph = StateGraph(AgentState)

enhanced_graph.add_node("agent", enhanced_call_model)
enhanced_graph.add_node("tools", enhanced_tool_node)

enhanced_graph.set_entry_point("agent")
enhanced_graph.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        END: END
    }
)
enhanced_graph.add_edge("tools", "agent")

# 그래프 컴파일
graph = enhanced_graph.compile()

print("✅ 향상된 멀티모달 에이전트 준비 완료!")
print("🔍 웹 검색 + 📄 문서 검색 + 📝 요약 기능 모두 사용 가능")

✅ 향상된 멀티모달 에이전트 준비 완료!
🔍 웹 검색 + 📄 문서 검색 + 📝 요약 기능 모두 사용 가능


In [81]:
def run_enhanced_agent(user_query: str) -> str:
    """향상된 에이전트 실행"""
    
    initial_state = {
        "messages": [HumanMessage(content=user_query)],
        "user_query": user_query,
        "search_results": [],
        "summary": "",
        "tool_calls": []
    }
    
    try:
        result = graph.invoke(initial_state)  # graph 사용 (이미 정의됨)
        final_message = result["messages"][-1]
        
        if hasattr(final_message, 'content'):
            return final_message.content
        else:
            return str(final_message)
            
    except Exception as e:
        return f"에이전트 실행 중 오류 발생: {str(e)}"

# pdf_agent는 graph로 대체
pdf_agent = graph

def interactive_pdf_agent():
    """대화형 PDF 에이전트"""
    print("\n" + "="*60)
    print("🤖 멀티모달 AI 어시스턴트")
    print("="*60)
    print("• '종료', 'quit', 'exit'를 입력하면 종료됩니다")
    print("• 예시 질문:")
    print("  - '교통약자에 대해 검색해줘'")
    print("  - '이 문서를 요약해줘'")
    print("  - '최신 AI 뉴스 검색해줘'")
    print("  - '문서 정보를 알려줘'")
    print("-"*60)
    
    while True:
        try:
            user_input = input("\n💬 질문을 입력하세요: ").strip()
            
            if user_input.lower() in ['종료', 'quit', 'exit', 'q']:
                print("에이전트를 종료합니다. 👋")
                break
            
            if not user_input:
                continue
            
            print("\n🤖 에이전트가 작업 중입니다...")
            print("-" * 40)
            
            # 향상된 에이전트 실행
            response = run_enhanced_agent(user_input)
            
            print(f"\n🎯 답변:\n{response}")
            print("-" * 60)
            
        except KeyboardInterrupt:
            print("\n\n에이전트를 종료합니다. 👋")
            break
        except Exception as e:
            print(f"오류 발생: {e}")

In [82]:
# # 벡터 스토어가 준비되었는지 확인
# if 'index' in locals() and index is not None:
#     print("✅ 멀티모달 AI 어시스턴트가 준비되었습니다!")
    
#     # 테스트 실행
#     test_query = "문서 정보를 알려줘"
#     print(f"\n🧪 테스트 실행: {test_query}")
#     result = run_enhanced_agent(test_query)
#     print(f"결과: {result[:200]}...")
    
#     # 대화형 실행
#     print("\n🚀 대화형 에이전트를 시작하시겠습니까? (y/n)")
#     if input("입력: ").lower() in ['y', 'yes', '네', 'ㅇ']:
#         interactive_pdf_agent()
        
# else:
#     print("❌ 벡터 스토어가 준비되지 않았습니다.")
#     print("먼저 PDF 처리와 벡터 스토어 생성을 완료해주세요.")

In [83]:
def run_safe_agent(user_query: str) -> str:
    """안전한 에이전트 실행 - 에러 방지"""
    
    try:
        # 간단한 직접 응답 케이스들
        if "안녕" in user_query or "hello" in user_query.lower():
            return "안녕하세요! 저는 멀티모달 AI 어시스턴트입니다. 문서 검색, 웹 검색, 요약 등을 도와드릴 수 있어요. 무엇을 도와드릴까요?"
        
        # 도구가 필요한 경우들
        if any(keyword in user_query for keyword in ["문서", "정보", "알려", "검색", "요약", "찾아"]):
            
            # 문서 정보 요청
            if "문서" in user_query and "정보" in user_query:
                return get_document_info().invoke({})
            
            # 문서 검색 요청  
            elif any(keyword in user_query for keyword in ["교통", "고령", "약자", "정책"]):
                if 'index' in globals() and index is not None:
                    return search_uploaded_documents.invoke({"query": user_query})
                else:
                    return "문서가 업로드되지 않았습니다. 먼저 PDF 파일을 업로드해주세요."
            
            # 웹 검색 요청
            elif any(keyword in user_query for keyword in ["최신", "뉴스", "날씨", "AI"]):
                return web_search.invoke({"query": user_query})
            
            # 요약 요청
            elif "요약" in user_query:
                if 'chunks' in globals() and chunks:
                    # 전체 문서의 일부를 요약
                    sample_content = "\n".join(chunks[:3])  # 처음 3개 청크
                    return summarize_content.invoke({"content": sample_content})
                else:
                    return "요약할 문서가 없습니다. 먼저 PDF 파일을 업로드해주세요."
        
        # 기본 LLM 응답
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
        response = llm.invoke([HumanMessage(content=user_query)])
        return response.content
        
    except Exception as e:
        return f"죄송합니다. 처리 중 오류가 발생했습니다: {str(e)}\n\n다른 질문을 시도해보시거나, 문서를 먼저 업로드해주세요."

In [84]:
# ============== Streamlit UI ===============
import streamlit as st
st.set_page_config(
    page_title="🤖 LangGraph AI 어시스턴트",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded"
)

def main():
    initialize_session_state()
    
    # 헤더
    st.markdown('<h1 style="text-align: center; color: #1f77b4;">🤖 LangGraph AI 어시스턴트</h1>', unsafe_allow_html=True)
    st.markdown('<p style="text-align: center; color: #666;">LangGraph Tools를 활용한 멀티모달 문서 어시스턴트</p>', unsafe_allow_html=True)
    
    # 사이드바
    with st.sidebar:
        st.header("📁 파일 업로드")
        
        uploaded_file = st.file_uploader(
            "PDF 파일을 선택하세요",
            type=['pdf'],
            help="PDF 파일을 업로드하면 LangGraph 도구들을 사용하여 문서를 분석할 수 있습니다."
        )
        
        if uploaded_file is not None:
            if st.button("파일 처리", type="primary"):
                success, message = process_file(uploaded_file)
                if success:
                    st.success(f"✅ {message}")
                else:
                    st.error(f"❌ {message}")
        
        # 상태 표시
        st.markdown("---")
        st.subheader("📊 시스템 상태")
        if st.session_state.file_processed:
            st.success("✅ 문서 처리 완료")
            st.info(f"📄 청크 수: {len(st.session_state.chunks)}")
            st.info(f"🔧 LangGraph: {'활성화' if st.session_state.agent_graph else '대기 중'}")
        else:
            st.warning("⏳ 문서 미처리")
        
        # LangGraph 도구 정보
        st.markdown("---")
        st.subheader("🛠️ 사용 가능한 도구들")
        st.markdown("""
        **LangGraph Tools:**
        - `search_uploaded_documents` 📄
        - `web_search_tool` 🌐  
        - `summarize_content_tool` 📝
        - `get_document_info_tool` ℹ️
        """)
        
        # 빠른 명령어
        st.markdown("---")
        st.subheader("🚀 빠른 테스트")
        
        if st.button("📄 문서 정보"):
            test_msg = {"role": "user", "content": "문서 정보를 알려줘"}
            st.session_state.messages.append(test_msg)
            response = run_agent_with_graph("문서 정보를 알려줘")
            st.session_state.messages.append({"role": "assistant", "content": response})
            st.rerun()
        
        if st.button("🔍 문서 검색"):
            test_msg = {"role": "user", "content": "교통약자에 대해 검색해줘"}
            st.session_state.messages.append(test_msg)
            response = run_agent_with_graph("교통약자에 대해 검색해줘")
            st.session_state.messages.append({"role": "assistant", "content": response})
            st.rerun()
    
    # 메인 채팅 영역
    st.header("💬 LangGraph 에이전트와 대화")
    
    # 채팅 기록
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
    
    # 채팅 입력
    if prompt := st.chat_input("LangGraph 도구들을 테스트해보세요..."):
        # 사용자 메시지
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)
        
        # 에이전트 응답
        with st.chat_message("assistant"):
            with st.spinner("LangGraph 에이전트가 도구들을 사용하여 응답 생성 중..."):
                response = run_agent_with_graph(prompt)
            st.markdown(response)
        
        st.session_state.messages.append({"role": "assistant", "content": response})
    
    # 기록 초기화
    if st.button("🗑️ 채팅 기록 초기화"):
        st.session_state.messages = []
        st.rerun()

if __name__ == "__main__":
    main()

2025-06-20 10:11:05.063 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 10:11:05.064 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 10:11:05.066 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-06-20 10:11:05.066 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 10:11:05.066 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 10:11:05.067 WARNING streamlit.runtime.scriptrunner_utils.script_run_c